# Binary Classifier: Waffles vs Pancakes

Run the code below to download the dataset `waffle_pancakes.zip`.

In [1]:
!gdown --id 1tJqtoF-5MNdC90rXYgtsXTs9o2odzEg5

Downloading...
From: https://drive.google.com/uc?id=1tJqtoF-5MNdC90rXYgtsXTs9o2odzEg5
To: /content/waffle_pancakes.zip
13.0MB [00:00, 60.3MB/s]


In [2]:
import os
import zipfile

local_zip = './waffle_pancakes.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [3]:
# Directories with our training images
train_waf_dir = os.path.join('./waffle_pancakes/train/waffles')
train_pan_dir = os.path.join('./waffle_pancakes/train/pancakes')

# Directories with our test images
test_waf_dir = os.path.join('./waffle_pancakes/test/waffles')
test_pan_dir = os.path.join('./waffle_pancakes/test/pancakes')

In [4]:
train_waf_names = os.listdir(train_waf_dir)
print(train_waf_names[:10])
train_pan_names = os.listdir(train_pan_dir)
print(train_pan_names[:10])

test_waf_names = os.listdir(test_waf_dir)
test_pan_names = os.listdir(test_pan_dir)

['img-waf-449.png', 'img-waf-494.png', 'img-waf-256.png', 'img-waf-483.png', 'img-waf-29.png', 'img-waf-169.png', 'img-waf-495.png', 'img-waf-175.png', 'img-waf-150.png', 'img-waf-319.png']
['img-pan-104.png', 'img-pan-179.png', 'img-pan-87.png', 'img-pan-165.png', 'img-pan-50.png', 'img-pan-186.png', 'img-pan-71.png', 'img-pan-366.png', 'img-pan-212.png', 'img-pan-237.png']


In [60]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
    ])

In [61]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)            

In [74]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
adam = Adam(lr=0.0005)

model.compile(loss='mse',
              optimizer=adam,
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [75]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      )

train_generator = train_datagen.flow_from_directory(
        './waffle_pancakes/train/',  # This is the source directory for training images
        batch_size=25,
        target_size=(150, 150),
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical',
        )

Found 947 images belonging to 2 classes.


In [76]:
history = model.fit(
      train_generator, 
      steps_per_epoch = 38, 
      epochs=10,
      verbose=1,
      validation_steps=8
      )

Epoch 1/10
38/38 [==============================] - 8s 193ms/step - loss: 0.1471 - accuracy: 0.7888
Epoch 2/10
38/38 [==============================] - 6s 169ms/step - loss: 0.1383 - accuracy: 0.7983
Epoch 3/10
38/38 [==============================] - 6s 169ms/step - loss: 0.1333 - accuracy: 0.8184
Epoch 4/10
38/38 [==============================] - 6s 168ms/step - loss: 0.1363 - accuracy: 0.8089
Epoch 5/10
38/38 [==============================] - 6s 168ms/step - loss: 0.1266 - accuracy: 0.8279
Epoch 6/10
38/38 [==============================] - 6s 169ms/step - loss: 0.1287 - accuracy: 0.8247
Epoch 7/10
38/38 [==============================] - 6s 170ms/step - loss: 0.1354 - accuracy: 0.8068
Epoch 8/10
38/38 [==============================] - 6s 170ms/step - loss: 0.1251 - accuracy: 0.8141
Epoch 9/10
38/38 [==============================] - 7s 172ms/step - loss: 0.1166 - accuracy: 0.8342
Epoch 10/10
38/38 [==============================] - 7s 171ms/step - loss: 0.1305 - accuracy: 0.8279

In [79]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))
  
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  print(classes)
  print("Pancake probability = " + str(classes[0][0]))
  print("Waffle prob = " + str(classes[0][1]))
  print()
  #if classes[0]<0.3:
  #  print(fn + " is a pancake")
  #elif classes[0]>0.7:
  #  print(fn + " is a waffle")
  #else:
  #  print("unsure")

Saving img-pan-test-51.png to img-pan-test-51 (1).png
[[5.0322044e-26 1.0000000e+00]]
Pancake probability = 5.0322044e-26
Waffle prob = 1.0

